In [1]:
from sklearn import metrics
import pandas as pd
import numpy as np
import glob
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.multiclass import OneVsRestClassifier
import time
import collections
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import defaultdict


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/My Drive/Colab Notebooks/plantrevista/
#!ls

In [3]:

def multi_clf_metrics(arq_csv):

    # Cria X (features) e y (labels)
    especies = pd.read_csv(arq_csv, low_memory=False)

    if arq_csv == 'resnet50_features.csv' or arq_csv == 'resnet152v2_features.csv':
      X = np.array(especies.iloc[0:,0:2048])
      y = np.array(especies.iloc[0:,2048])		
    if arq_csv == 'nasnetlarge_features.csv':
      X = np.array(especies.iloc[0:,0:4032])
      y = np.array(especies.iloc[0:,4032])		
    if arq_csv == 'plantclef2013_inception_V3.csv':
      X = np.array(especies.iloc[0:,0:2048])
      y = np.array(especies.iloc[0:,2048])		
        
    # Executa o PCA
    
    pca = PCA(.90)
    principal_components = pca.fit_transform(X)

    classifiers = {
        'SVC - (polynomial, ovr)': OneVsRestClassifier(svm.SVC(kernel='poly', C=0.1, gamma=0.0001,degree=3, coef0=0.0,
                                                               shrinking=True, probability=False, tol=0.001, cache_size=2000,
                                                               class_weight=None, verbose=False, max_iter=-1,
                                                               decision_function_shape='ovr', random_state=None))
    }
    scoring = ['accuracy',
          'precision_macro','recall_macro','f1_macro',
          'precision_micro','recall_micro','f1_micro',
          'precision_weighted','recall_weighted','f1_weighted']
    for name_clf, clf in classifiers.items():
      
      inicio = time.time()
      
      #Variáveis do Dataset de Resultados
      arquivo_csv = []
      classifier = []
      accuracy = []
      std_accuracy = []

      # Metrics Macro
      precision_macro = []
      std_precision_macro = []
      recall_macro = []
      std_recall_macro = []
      f1_macro = []
      std_f1_macro = []

      # Metrics Micro
      precision_micro = []
      std_precision_micro = []
      recall_micro = []
      std_recall_micro = []
      f1_micro = []
      std_f1_micro = []

      # Metrics Weighted
      precision_weighted = []
      std_precision_weighted = []
      recall_weighted = []
      std_recall_weighted = []
      f1_weighted = []
      std_f1_weighted = []

      # Informações do PCA
      n_samples = []
      princ_compo = []
      variancia = []

      print(f'Executando classificador \033[1;31m{name_clf}\033[m')
      classifier.append(name_clf)
      arquivo_csv.append(arq_csv[:-4])
      scores = cross_validate(clf, principal_components, y, scoring=scoring, cv=10, return_train_score=False,n_jobs=-1, verbose=51)

      accuracy.append(scores['test_accuracy'].mean())
      std_accuracy.append(scores['test_accuracy'].std())

      # Metrics Macro
      precision_macro.append(scores['test_precision_macro'].mean())
      std_precision_macro.append(scores['test_precision_macro'].std())
      recall_macro.append(scores['test_recall_macro'].mean())
      std_recall_macro.append(scores['test_recall_macro'].std())
      f1_macro.append(scores['test_f1_macro'].mean())
      std_f1_macro.append(scores['test_f1_macro'].std())
      print(f'\033[1;32mMétricas Macro extraidas!\033[m')

      # Metrics Micro
      precision_micro.append(scores['test_precision_micro'].mean())
      std_precision_micro.append(scores['test_precision_micro'].std())
      recall_micro.append(scores['test_recall_micro'].mean())
      std_recall_micro.append(scores['test_recall_micro'].std())
      f1_micro.append(scores['test_f1_micro'].mean())
      std_f1_micro.append(scores['test_f1_micro'].std())
      print(f'\033[1;32mMétricas Micro extraidas!\033[m')

      # Metrics Weighted
      precision_weighted.append(scores['test_precision_weighted'].mean())
      std_precision_weighted.append(scores['test_precision_weighted'].std())
      recall_weighted.append(scores['test_recall_weighted'].mean())
      std_recall_weighted.append(scores['test_recall_weighted'].std())
      f1_weighted.append(scores['test_f1_weighted'].mean())
      std_f1_weighted.append(scores['test_f1_weighted'].std())
      print(f'\033[1;32mMétricas Weighted extraidas!\033[m')

      n_samples.append(principal_components.shape[0])
      princ_compo.append(principal_components.shape[1])
      variancia.append(sum(pca.explained_variance_ratio_))

      print(f'Base {arq_csv[:-4]} \033[1;32mOK!\033[m')
      print('=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=')
      print('')
      
      # Criando um DataFrame com os Resultados
      data = {'Dataset': arquivo_csv, 'Classificador': classifier, 'Accuracy': accuracy,'D.P. - Acurácia': std_accuracy, 
          'Precision Macro': precision_macro, 'D.P - Precision Macro': std_precision_macro,
          'Recall Macro': recall_macro, 'D.P - Recall Macro': std_recall_macro,
          'F1 Macro': f1_macro, 'D.P - F1 Macro': std_f1_macro,
          'Precision Micro': precision_micro, 'D.P - Precision Micro': std_precision_micro,
          'Recall Micro': recall_micro, 'D.P - Recall Micro': std_recall_micro,
          'F1 Micro': f1_micro, 'D.P - F1 Micro': std_f1_micro,
          'Precision Weighted': precision_weighted, 'D.P - Precision Weighted': std_precision_weighted,
          'Recall Weighted': recall_weighted, 'D.P - Recall Weighted': std_recall_weighted,
          'F1 Weighted': f1_weighted, 'D.P - F1 Weighted': std_f1_weighted,
          'N. Amostras': n_samples, 'PCA': princ_compo, 'Variância': variancia}
      df = pd.DataFrame(data)
      df.to_csv('resultados/'+name_clf+'_'+arq_csv[:-4]+'.csv', index=None)
      
      fim = time.time()
      print(f'Tempo de Execução: {(fim - inicio)/3600} horas')
      print('')

In [4]:
multi_clf_metrics('resnet152v2_features.csv')


Executando classificador SVC - (polynomial, ovr)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
Memmapping (shape=(26077, 274), dtype=float64) to new file /var/folders/k2/7m7wfg6n6y5fylg_14w25lnc0000gp/T/joblib_memmapping_folder_24051_6741672734/24051-4344023416-4e95f6eb4c6f4e179172126a4c01c1dd.pkl
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 22.9min
[Parallel(n_jobs=-1)]: Done   2 out of  10 | elapsed: 23.0min remaining: 92.1min
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed: 23.1min remaining: 53.9min
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed: 23.2min remaining: 34.7min
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed: 23.2min remaining: 23.2min
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed: 23.3min remaining: 15.5min
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed: 23.3min remaining: 10.0min
[Parallel(n_jobs=-1)]: Done   8 out of  10 | elapsed: 23.3min remaining:  5.8min
[Parallel(n_jobs=-1)]: Done  10 out of  10